In [373]:
import csv 
import numpy


def levenshteinDistanceDP(token1, token2):
    
    '''
    I. 
    
    Creates a matrix of y-dim by x=dim:  
    
    distances=numpy.zeros(y-dim, x-dim)
    distances[y-coordinate][x-coordinate]. 
    '''
    
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1)) 

    '''
    II. Creates numeric labels for matrix columns and rows.
    '''
    
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
    
        
        
    a = 0
    b = 0
    c = 0
    
    
    
    '''
    III.  
    
    Plots in zeros for characters that are identical, 
    where distances[1][1] representes the value for the first character of both tokens,  
    distances[2][2] represents the value for the second character of both tokens, 
    distances[1][0] reoresents the score for the first character of token 1 compared to a non-existent token2, 
    etc.
    
    Iterates through each of the letters of the first token, and calculates LD score for all those letters combined 
    with only the first letter of the second token, and then repeats the same thing until 
    scores have been calculated for each of characters in the second token. 
    
    '''
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):                #If characters are the same, score is 0.
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            elif (token1[t1-1], token2[t2-1]) == ('p', 'd'):  #If characters meet our criteria, score is x.
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                if (a <= b and a <= c):
                    distances[t1][t2] = a + .5
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + .5
                else:
                    distances[t1][t2] = c + .5
            else:                                             #For all other characters, score is 1.
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]

                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    print((distances, len(token1), len(token2)))
    return distances[len(token1)][len(token2)]
                
  

In [376]:
levenshteinDistanceDP('pig', 'dig')

(array([[0. , 1. , 2. , 3. ],
       [1. , 0.5, 1.5, 2.5],
       [2. , 1.5, 0.5, 1.5],
       [3. , 2.5, 1.5, 0.5]]), 3, 3)


0.5

In [378]:
levenshteinDistanceDP('pop', 'dod')

(array([[0. , 1. , 2. , 3. ],
       [1. , 0.5, 1.5, 2. ],
       [2. , 1.5, 0.5, 1.5],
       [3. , 2. , 1.5, 1. ]]), 3, 3)


1.0

In [379]:
levenshteinDistanceDP('pip', 'dod')

(array([[0. , 1. , 2. , 3. ],
       [1. , 0.5, 1.5, 2. ],
       [2. , 1.5, 1.5, 2.5],
       [3. , 2. , 2.5, 2. ]]), 3, 3)


2.0

In [380]:
levenshteinDistanceDP('tig', 'dod')

(array([[0., 1., 2., 3.],
       [1., 1., 2., 3.],
       [2., 2., 2., 3.],
       [3., 3., 3., 3.]]), 3, 3)


3.0

In [284]:
import csv 
import numpy


def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    #printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

def LDScore(infile, outfile): 

    with open(infile, 'r', encoding='utf-8') as source, open(outfile, 'w', encoding='utf-8') as sink: 
        reader = csv.reader(source, delimiter = ',')
        writer = csv.writer(sink, delimiter = '\t')
        count = 0
        for line in reader: 
            ASRAns = line[0].rstrip()
            ExpAns = line[1].rstrip()
            LD = levenshteinDistanceDP(ASRAns, ExpAns)
            writer.writerow((ASRAns, ExpAns, LD))


In [262]:
LDScore('ASREs.IPA.txt', 'ASREs.LD')

In [263]:
import pandas as pd
df = pd.read_csv('ASREs.txt', encoding='utf-8', sep='\t')
df.columns = ['ASR Ans', 'Exp Ans', 'LD score']
df_sorted = df.sort_values(by='LD score')



characters = {}
for ASR, Exp, LD in zip(df_sorted['ASR Ans'].head(6), df_sorted['Exp Ans'].head(6), df_sorted['LD score'].head(6)): 
    Chars = []
    for ASRChar, ExpChar in enumerate(zip(ASR, Exp)):
            Chars.append((ASRChar, ExpChar))
        
    characters[ASR, Exp, LD] = Chars

different_phones = {}
for k,v in characters.items(): 
    diff_phones = []
    for item in v: 
        if item[1][0]!=item[1][1]:
            diff_phones.append(item)
    
    different_phones[k] = diff_phones
            

print(different_phones)

{('beðo', 'deðo', 1.0): [(0, ('b', 'd'))], ('keðo', 'deðo', 1.0): [(0, ('k', 'd'))], ('peðo', 'deðo', 1.0): [(0, ('p', 'd'))], ('deɾo', 'deðo', 1.0): [(2, ('ɾ', 'ð'))], ('teto', 'deðo', 2.0): [(0, ('t', 'd')), (2, ('t', 'ð'))], ('bweno', 'deðo', 3.0): [(0, ('b', 'd')), (1, ('w', 'e')), (2, ('e', 'ð')), (3, ('n', 'o'))]}


In [264]:
EsConFeatures = {}

In [270]:
EsConFeatures['n'] = set(('nasal', 'alveolar', 'voiced'))
EsConFeatures['f'] = set(('fricative', 'labiodental', 'voiceless'))
EsConFeatures['m'] = set(('nasal', 'bilabial', 'voiced'))
EsConFeatures['b'] = set(('plosive', 'bilabial', 'voiceless'))
EsConFeatures['ɾ'] = set(('tap', 'alveolar','voiced'))
EsConFeatures['k'] = set(('plosive', 'velar', 'voiceless'))
EsConFeatures['l'] = set(('lateral approximant', 'alveolar', 'voiced'))
EsConFeatures['d'] = set(('plosive','alveolar','voiced'))
EsConFeatures['v'] = set(('fricative', 'labiodental','voiceless'))
EsConFeatures['p'] = set(('plosive', 'bilabial','voiced'))
EsConFeatures['j'] = set(('approximant', 'palatal','voiced'))
EsConFeatures['ð'] = set(('fricative', 'dental','voiced'))
EsConFeatures['w'] = set(('approximant','bilabial','voiced'))
EsConFeatures['ʃ'] = set(('fricative', 'post-alveolar','voiceless'))
EsConFeatures['t'] = set(('plosive', 'alveolar', 'voiceless'))
EsConFeatures['ɡ'] = set(('plosive', 'velar', 'voiced'))
EsConFeatures['ŋ'] = set(('nasal', 'velar','voiced'))
EsConFeatures['s'] = set(('fricative','alveolar', 'voiceless'))
#EsConFeatures['d͡ʒ'] = set(('affricate', 'post-alveolar', 'voiced'))
#EsConFeatures['t͡ʃ'] = set(('affricate', 'post-alveolar', 'voiceless'))
EsConFeatures['β'] = set(('fricative' 'bilabial', 'voiced'))
EsConFeatures['x'] = set(('fricative', 'velar', 'voiceless'))
EsConFeatures['ɣ'] = set(('fricative', 'velar', 'voiced'))
EsConFeatures['ʝ'] = set(('fricative', 'palatal', 'voiced'))
EsConFeatures['ʎ'] = set(('lateral approximant', 'palatal', 'voiced'))

In [266]:

minimal_pairs = set()
TwoFeature_pairs = set()
for k,v in different_phones.items(): 
    print(k,v)
    for item in v: 
        ASRChar = item[1][0]
        ExpChar = item[1][1]
        position = item[0]
        if position == 0:
            if len(EsConFeatures[ASRChar]-EsConFeatures[ExpChar])==1: 
                minimal_pairs.add(
                    (k[0], k[1])
                )
            if len(EsConFeatures[ASRChar]-EsConFeatures[ExpChar])==2:
                TwoFeature_pairs.add(
                    (k[0], k[1])
                )

print(f'''
minimal_pairs: {minimal_pairs}
TwoFeature_pairs: {TwoFeature_pairs}
''')

('beðo', 'deðo', 1.0) [(0, ('b', 'd'))]
('keðo', 'deðo', 1.0) [(0, ('k', 'd'))]
('peðo', 'deðo', 1.0) [(0, ('p', 'd'))]
('deɾo', 'deðo', 1.0) [(2, ('ɾ', 'ð'))]
('teto', 'deðo', 2.0) [(0, ('t', 'd')), (2, ('t', 'ð'))]
('bweno', 'deðo', 3.0) [(0, ('b', 'd')), (1, ('w', 'e')), (2, ('e', 'ð')), (3, ('n', 'o'))]

minimal_pairs: {('teto', 'deðo'), ('peðo', 'deðo')}
TwoFeature_pairs: {('beðo', 'deðo'), ('keðo', 'deðo'), ('bweno', 'deðo')}



In [267]:
for index, row in df_sorted.iterrows():
    if (row['ASR Ans'], row['Exp Ans']) in minimal_pairs: 
        df_sorted.loc[index, 'LD score']=row['LD score']-.5
    if (row['ASR Ans'], row['Exp Ans']) in TwoFeature_pairs: 
        df_sorted.loc[index, 'LD score']=row['LD score']-.25
    else: 
        continue
 

      


In [269]:
df_sorted.head(6)

,ASR Ans,Exp Ans,LD score
2,beðo,deðo,0.75
3,keðo,deðo,0.75
5,peðo,deðo,0.50
9,deɾo,deðo,1.00
4,teto,deðo,1.50
30,bweno,deðo,2.75


In [ ]:
# vowels have 4 features: high/low,front/back, tense/lax (long/short), lip rounding +/-

# 1. check all consonants for mistakes 
# 2.  try to understand current LD function, or find another one that will allow you to understand what it's doing.  
# 3.  do a feature set for vowels
    